In [1]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import *
from settings import obtener_timestamp
import time

In [2]:
""" Configuramos Spark """
conf = SparkConf()
conf.setAppName("ProcesamientoDatos")
conf.setMaster("local[4]")

In [3]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [4]:
data = spark.read.format('parquet').load('pruebas.parquet/')

In [5]:
tiempoFin = obtener_timestamp("2013-01-01", "00:30")
tiempo30 = tiempoFin - timedelta(minutes=30)
tiempo15 = tiempoFin - timedelta(minutes=15)

### Consulta 2: Zonas que más beneficios pueden procurar

En esta segunda búsqueda lo que vamos a obtener es las 10 zonas que más beneficios pueden generar en el momento de la búsqueda (hora introducida). La cantidad de benficio de una zona consigue dividiendo el beneficio de una zona por la catidad de taxis vacios que hay en la zona.

El beneficio de una zona es la media de la suma de la tarifa y la propina de los viajes que han acabado en los últimos 15 minutos y empezaron en esa zona. 

Por otro lado, para el número de taxis, se tendrán en cuenta el número de taxis vacíos en una zona durante los 30 minutos anteriores. 

    hora_subida, hora_bajada, area_mas_beneficiosa_1, ..., area_mas_beneficiosa_10, tiempo_ejecucion

Vamos a empezar con los taxis vacíos en una zona.

- Primero filtraremos los viajes acabados en la última media hora y nos quedamos con el último viaje acabado.
- Ahora buscamos el último viaje empezado por los taxis en esa franja de tiempo

Prueba de que nos quedamos con el último viaje acabado de cada taxi, eliminando si ha hecho más de un viaje.

In [6]:
tripsDown = data.where(data.hora_bajada <= tiempoFin).where(data.hora_bajada > tiempo30)

In [7]:
trips30 = tripsDown.select("medallon", "hora_bajada", "cuad_latitud_bajada", "cuad_longitud_bajada").orderBy(["medallon","hora_bajada"], ascending=[1,0]) \
    

In [8]:
trips40 = tripsDown.select("medallon", "hora_bajada", "cuad_latitud_bajada", "cuad_longitud_bajada").orderBy("hora_bajada", ascending=False) \
    .dropDuplicates(subset=["medallon"]).orderBy(["medallon","hora_bajada"], ascending=[1,0])

In [9]:
trips30.show()
trips40.show()

+--------------------+--------------------+-------------------+--------------------+
|            medallon|         hora_bajada|cuad_latitud_bajada|cuad_longitud_bajada|
+--------------------+--------------------+-------------------+--------------------+
|07290D3599E7A0D62...|2013-01-01 00:02:...|                170|                 160|
|0EC22AAF491A8BD91...|2013-01-01 00:25:...|                160|                 159|
|120E700FE35B2DDBE...|2013-01-01 00:05:...|                163|                 156|
|1E5F4C1CAE7AB3D06...|2013-01-01 00:26:...|                160|                 159|
|C306CAC565429C128...|2013-01-01 00:28:...|                162|                 155|
|C306CAC565429C128...|2013-01-01 00:04:...|                152|                 165|
+--------------------+--------------------+-------------------+--------------------+

+--------------------+--------------------+-------------------+--------------------+
|            medallon|         hora_bajada|cuad_latitud_bajada|c

In [10]:
tripsDown

DataFrame[medallon: string, licencia: string, hora_subida: timestamp, hora_bajada: timestamp, duracion_viaje_seg: int, longitud_subida: decimal(18,14), latitud_subida: decimal(18,14), longitud_bajada: decimal(18,14), latitud_bajada: decimal(18,14), tipo_pago: string, tarifa: decimal(10,2), propina: decimal(10,2), cantidad_total: decimal(10,2), cuad_latitud_subida: bigint, cuad_longitud_subida: bigint, cuad_latitud_bajada: bigint, cuad_longitud_bajada: bigint]

Vamos a coger ahora el último viaje iniciado

In [11]:
tripsUp = data.where(data.hora_subida <= tiempoFin).where(data.hora_subida > tiempo30)

In [12]:
tripUp = tripsUp.select("medallon", "hora_subida").orderBy(["medallon","hora_subida"], ascending=[True,False])

In [13]:
tripsUp30 = tripsUp.select("medallon", "hora_subida").orderBy(["medallon","hora_subida"], ascending=[True,False]) \
    .dropDuplicates(subset=["medallon"]).orderBy(["medallon","hora_subida"], ascending=[1,0])

In [14]:
tripUp.show()
tripsUp30.show()

+--------------------+--------------------+
|            medallon|         hora_subida|
+--------------------+--------------------+
|07290D3599E7A0D62...|2013-01-01 00:03:...|
|07290D3599E7A0D62...|2013-01-01 00:01:...|
|0EC22AAF491A8BD91...|2013-01-01 00:01:...|
|1E5F4C1CAE7AB3D06...|2013-01-01 00:03:...|
|C306CAC565429C128...|2013-01-01 00:05:...|
|C306CAC565429C128...|2013-01-01 00:01:...|
+--------------------+--------------------+

+--------------------+--------------------+
|            medallon|         hora_subida|
+--------------------+--------------------+
|07290D3599E7A0D62...|2013-01-01 00:03:...|
|0EC22AAF491A8BD91...|2013-01-01 00:01:...|
|1E5F4C1CAE7AB3D06...|2013-01-01 00:03:...|
|C306CAC565429C128...|2013-01-01 00:05:...|
+--------------------+--------------------+



In [15]:
bueno30Down = tripsDown.select("medallon", "hora_bajada", "cuad_latitud_bajada", "cuad_longitud_bajada") \
    .orderBy("hora_bajada", ascending=False) \
    .dropDuplicates(subset=["medallon"])
bueno30Up = tripsUp.select("medallon", "hora_subida").orderBy("hora_subida", ascending=False) \
    .dropDuplicates(subset=["medallon"]).withColumnRenamed("medallon", "taxi")

In [16]:
bueno30Down.show()
bueno30Up.show()

+--------------------+--------------------+-------------------+--------------------+
|            medallon|         hora_bajada|cuad_latitud_bajada|cuad_longitud_bajada|
+--------------------+--------------------+-------------------+--------------------+
|C306CAC565429C128...|2013-01-01 00:28:...|                162|                 155|
|07290D3599E7A0D62...|2013-01-01 00:02:...|                170|                 160|
|1E5F4C1CAE7AB3D06...|2013-01-01 00:26:...|                160|                 159|
|120E700FE35B2DDBE...|2013-01-01 00:05:...|                163|                 156|
|0EC22AAF491A8BD91...|2013-01-01 00:25:...|                160|                 159|
+--------------------+--------------------+-------------------+--------------------+

+--------------------+--------------------+
|                taxi|         hora_subida|
+--------------------+--------------------+
|C306CAC565429C128...|2013-01-01 00:05:...|
|07290D3599E7A0D62...|2013-01-01 00:03:...|
|1E5F4C1CAE7AB

In [17]:
#spark falla join si el nombre de la columna es el mismo, renombrado a taxi
joined = bueno30Down.join(bueno30Up, bueno30Down.medallon == bueno30Up.taxi, "leftouter") \
    .select("medallon", "hora_bajada", "hora_subida", "cuad_latitud_bajada", "cuad_longitud_bajada")

In [18]:
joined.show()

+--------------------+--------------------+--------------------+-------------------+--------------------+
|            medallon|         hora_bajada|         hora_subida|cuad_latitud_bajada|cuad_longitud_bajada|
+--------------------+--------------------+--------------------+-------------------+--------------------+
|C306CAC565429C128...|2013-01-01 00:28:...|2013-01-01 00:05:...|                162|                 155|
|07290D3599E7A0D62...|2013-01-01 00:02:...|2013-01-01 00:03:...|                170|                 160|
|1E5F4C1CAE7AB3D06...|2013-01-01 00:26:...|2013-01-01 00:03:...|                160|                 159|
|120E700FE35B2DDBE...|2013-01-01 00:05:...|                null|                163|                 156|
|0EC22AAF491A8BD91...|2013-01-01 00:25:...|2013-01-01 00:01:...|                160|                 159|
+--------------------+--------------------+--------------------+-------------------+--------------------+



In [19]:
pruebasubidas = joined.select(joined.medallon, joined.cuad_latitud_bajada, joined.cuad_longitud_bajada, \
    when(joined.hora_subida < tiempoFin, 1).otherwise(0).alias("taxi_en_uso"), \
    when(joined.hora_subida > joined.hora_bajada, 1).otherwise(0).alias("taxi_ocupado"))

In [25]:
estado_taxis = joined.select(joined.medallon, joined.cuad_latitud_bajada, joined.cuad_longitud_bajada, \
    when(joined.hora_subida > joined.hora_bajada, 1).otherwise(0).alias("taxi_ocupado"))

In [26]:
estado_taxis.show()

+--------------------+-------------------+--------------------+--------------------+------------+
|            medallon|cuad_latitud_bajada|cuad_longitud_bajada|         hora_bajada|taxi_ocupado|
+--------------------+-------------------+--------------------+--------------------+------------+
|C306CAC565429C128...|                162|                 155|2013-01-01 00:28:...|           0|
|07290D3599E7A0D62...|                170|                 160|2013-01-01 00:02:...|           1|
|1E5F4C1CAE7AB3D06...|                160|                 159|2013-01-01 00:26:...|           0|
|120E700FE35B2DDBE...|                163|                 156|2013-01-01 00:05:...|           0|
|0EC22AAF491A8BD91...|                160|                 159|2013-01-01 00:25:...|           0|
+--------------------+-------------------+--------------------+--------------------+------------+



In [27]:
taxis_filtrados = estado_taxis.filter(estado_taxis.taxi_ocupado == 0)

In [28]:
pruebasubidas.show()
taxis_filtrados.show()

+--------------------+-------------------+--------------------+-----------+------------+
|            medallon|cuad_latitud_bajada|cuad_longitud_bajada|taxi_en_uso|taxi_ocupado|
+--------------------+-------------------+--------------------+-----------+------------+
|C306CAC565429C128...|                162|                 155|          1|           0|
|07290D3599E7A0D62...|                170|                 160|          1|           1|
|1E5F4C1CAE7AB3D06...|                160|                 159|          1|           0|
|120E700FE35B2DDBE...|                163|                 156|          0|           0|
|0EC22AAF491A8BD91...|                160|                 159|          1|           0|
+--------------------+-------------------+--------------------+-----------+------------+

+--------------------+-------------------+--------------------+--------------------+------------+
|            medallon|cuad_latitud_bajada|cuad_longitud_bajada|         hora_bajada|taxi_ocupado|
+-

In [29]:
taxis_libres = taxis_filtrados.groupBy("cuad_latitud_bajada", "cuad_longitud_bajada").count() \
    .select(col("cuad_latitud_bajada"), col("cuad_longitud_bajada"), col("count").alias("taxis_libres"))

In [30]:
taxis_libres.show()

+-------------------+--------------------+------------+
|cuad_latitud_bajada|cuad_longitud_bajada|taxis_libres|
+-------------------+--------------------+------------+
|                163|                 156|           1|
|                162|                 155|           1|
|                160|                 159|           2|
+-------------------+--------------------+------------+



### Beneficio zona

- Acotamos a viajes de los últimos 15 min
- Agrupamos por zona de inicio del viaje
- Calculamos la media de la tarifa y de la propina de cada zona
- Sumamos cada media y obtenemos el beneficio medio de las zonas

In [113]:
trips15 = tripsDown.where(data.hora_bajada <= tiempoFin).where(data.hora_bajada > tiempo15)

In [126]:
beneficios = trips15.groupBy("cuad_latitud_subida", "cuad_longitud_subida") \
    .avg("tarifa", "propina") \
    .select(col("cuad_latitud_subida"), col("cuad_longitud_subida"), \
        (col("avg(tarifa)") + col("avg(propina)")).alias("beneficios"))

In [127]:
beneficios.show()

+-------------------+--------------------+----------+
|cuad_latitud_subida|cuad_longitud_subida|beneficios|
+-------------------+--------------------+----------+
|                162|                 158|  4.000000|
|                162|                 155|  2.500000|
|                152|                 165|  4.500000|
+-------------------+--------------------+----------+



In [128]:
beneficios.printSchema()
taxis_libres.printSchema()

root
 |-- cuad_latitud_subida: long (nullable = true)
 |-- cuad_longitud_subida: long (nullable = true)
 |-- beneficios: decimal(15,6) (nullable = true)

root
 |-- cuad_latitud_bajada: long (nullable = true)
 |-- cuad_longitud_bajada: long (nullable = true)
 |-- taxis_libres: long (nullable = false)



Aquí unimos ambas tablas, sumando 1 a los taxis libres para evitar la división entre 0 y sumar la repartición de beneficios que tendría que hacer el taxista con el compañero

In [137]:
condicion = [beneficios.cuad_latitud_subida == taxis_libres.cuad_latitud_bajada, \
             beneficios.cuad_longitud_subida == taxis_libres.cuad_longitud_bajada]
profitable = beneficios.join(taxis_libres, condicion, "leftouter") \
    .select(col("cuad_latitud_subida").alias("cuad_latitud"), \
            col("cuad_longitud_subida").alias("cuad_longitud"), (col("beneficios") / \
            when(taxis_libres.taxis_libres > 0, (taxis_libres.taxis_libres + 1)).otherwise(1)).alias("beneficio")) \
    .orderBy("beneficio", ascending=False)

In [138]:
profitable.show()

+------------+-------------+--------------------+
|cuad_latitud|cuad_longitud|           beneficio|
+------------+-------------+--------------------+
|         152|          165|4.500000000000000...|
|         162|          158|4.000000000000000...|
|         162|          155|1.250000000000000...|
+------------+-------------+--------------------+



In [136]:
profitable.printSchema()

root
 |-- cuad_latitud: long (nullable = true)
 |-- cuad_longitud: long (nullable = true)
 |-- Beneficio: decimal(36,27) (nullable = true)

